# Bethe-Salpeter equation in QTCI representation with patches (############## TODO: QTCI or QTT?)

## Bethe-Salpeter equation

Our objective in this work is computing the Bethe-Salpeter equation (BSE), which relates the full vertex $F^r$, the bare susceptibility ${\Chi_0}^r$ and the channel-irreducible vertex $\Gamma^r$
$$
    F^r_{\nu\nu'\omega} = \Gamma^r_{\nu\nu'\omega} + \frac{1}{\beta^2}\sum_{\nu''\nu'''} F^r_{\nu\nu''\omega} {\Chi_0}^r_{\nu''\nu'''\omega} \Gamma^r_{\nu'''\nu'\omega}.
$$
The sum term is equal to the channel-reducible vertex $\Phi^r \equiv F^r_{\nu\nu'\omega} - \Gamma^r_{\nu\nu'\omega}$.
Here, $r$ denotes one of four (density $r=d$, magnetic $r=m$, singlet $r=s$, triplet $r=t$) channels, $\nu$ (and its primed variants) a fermionic Matsubara frequency (of the form $(2n+1)\pi/\beta$ for some integer $n$) and $\omega$ a bosonic Matsubara frequency (of the form $2m\pi/\beta$ for some integer $m$).
The sums are performed over all fermionic frequencies and $\beta = 1/T$ is inverse temperature.

If we chose to straightforwardly represent our quantities by sampling them in a box centered on the origin, the BSE would be easy to implement: For each bosonic frequency, two matrix-matrix multiplications would suffice.
But as this strategy sooner rather than later hits a wall in terms of computing power and memory requirements when approaching lower temperatures, where a larger frequency box is required to yield the same accuracy target in the BSE, people have been looking for alternative solutions for some time. (############## TODO: sources)

## Quantics tensor trains

In the present work, we utilize a quantics tensor train (QTT) representation [@Shinaoka2023].
A function of a single Matsubara frequency $f_\nu$ can be approximately represented by its values on a $2^R$-frequency mesh $\nu \in \{\pi/\beta, 3\pi/\beta, \ldots, (2^{R+1} - 1)\pi/\beta\}$ (in practice, the frequency box is usually centered on the origin) where $R \in \mathbb{N}$ controls the accuracy of the representation.
Each such $\nu$ can be written as
$$
    \nu = \left(\nu_1 2^R + \nu_2 2^{R-1} + \ldots + \nu_R + 1\right)\pi/\beta
$$
where the $\nu_r \in \{0, 1\}$ are binary variables.
Thus $f$ can be viewed as an order-$R$ tensor,
$$
    f_\nu = f_{\nu_1\nu_2\ldots\nu_R}
$$
which then allows us to perform SVD and convert it into a tensor train (also known as multi-particle state or MPS)
$$
    f_{\nu_1\nu_2\ldots\nu_R} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{R-1}=1}^{D_{R-1}} f^{(1)}_{\nu_1,1\alpha_1} f^{(2)}_{\nu_2,\alpha_1\alpha_2} \cdots f^{(R)}_{\nu_R,\alpha_{R-1}1}. 
$$
Here, $f^{(r)}_{\nu_r,\alpha_{r-1}\alpha_{r}}$ is a $2 \times D_{r-1} \times D_r$ tensor and $D_r$ is the bond dimension between two neighboring tensors.
We call $\alpha_r$ the bond (or internal) indices, $\nu_r$ the local (or external) indices and $D = \max_r D_r$ the tensor train's bond dimension.
If it is high enough, $D \sim 2^R$, the tensor is represented exactly, but to compress the original function, we can truncate the tensor and throw away unimportant information.
We will not go into the details of how to find the tensor train representation here, but suffice it to say we employ Tensor cross interpolation (TCI) [@Ritter2024].

### Multi-particle operators

A very similar concept is that of an multi-particle operator (MPO), which consists of order 4 tensors and has two external indices per tensor.
As the name implies, an MPO can be multiplied onto an MPS (or another MPO), being analogous to a matrix if MPSs are viewed as analogous to vectors:
$$
\begin{aligned}
    (A \cdot f)_{\nu_1\ldots\nu_R} &= \sum_{\nu_1',\ldots,\nu_R'} A^{(1),\nu_1'}_{\nu_1} \cdot \ldots \cdot A^{(R),\nu_R'}_{\nu_R} f^{(1)}_{\nu_1'} \cdot \ldots \cdot f^{(R)}_{\nu_R'} \\
    &= \sum_{\nu_1',\ldots,\nu_R'} \sum_{\alpha_{A,1},\ldots,\alpha_{A,R}} \sum_{\alpha_{f,1},\ldots,\alpha_{f,R}} A^{(1),\nu_1'}_{\nu_1,1\alpha_{A,1}} f^{(1)}_{\nu_1',1\alpha_{f,1}} \cdots A^{(R),\nu_R'}_{\nu_R,\alpha_{A,R}1} f^{(R)}_{\nu_R',\alpha_{f,R}1}
\end{aligned}
$$

### Multiple variables

To form the MPS of a multivariate function $f_{\nu\nu'}$, we have two choices:

- Interleaved representation: $2R$ tensors of dimension $2 \times D_{r-1} \times D_r$.
    $$
        f_{\nu\nu'} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{2R-1}=1}^{D_{2R-1}} f^{(1)}_{\nu_1,1\alpha_1} f^{(2)}_{\nu_1',\alpha_1\alpha_2} f^{(3)}_{\nu_2,\alpha_2\alpha_3} f^{(4)}_{\nu_2',\alpha_3\alpha_4} \cdots f^{(2R-1)}_{\nu_R,\alpha_{2R-2}\alpha_{2R-1}} f^{(2R)}_{\nu_R',\alpha_{2R-1}1}
    $$
- Fused representation: $R$ tensors of dimension $4 \times D_{r-1} \times D_r$.
    $$
        f_{\nu\nu'} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{R-1}=1}^{D_{R-1}} f^{(1)}_{(\nu_1,\nu_1'),1\alpha_1} f^{(2)}_{(\nu_2,\nu_2'),\alpha_1\alpha_2} \cdots f^{(R)}_{(\nu_R,\nu_R'),\alpha_{R-1}1}
    $$

In the following, we will work in the fused representation.

### Multiplication of MPSs

For clarity, we will use $i$, $j$ and $k$ in this section instead of $\nu$, $\nu'$ and $\nu''$.
To compute
$$
    h_{ij} = \sum_k f_{ik} g_{kj},
$$
we introduce an auxiliary MPO $\hat{f}$ such that
$$
    h_{ij} = \sum_{kl} \hat{f}^{kl}_{ij} g_{kl}
$$
which is given by
$$
    \hat{f}^{kl}_{ij} = \sum_{\alpha_1,\ldots,\alpha_{R_1}} \hat{f}^{(1),(k_1, l_1)}_{(i_1, j_1),1\alpha_1} \hat{f}^{(2),(k_2, l_2)}_{(i_2, j_2),\alpha_1\alpha_2} \cdots \hat{f}^{(R),(k_R, l_R)}_{(i_R, j_R),\alpha_{R-1}1}
$$
with
$$
    \hat{f}^{(r),(k_r, l_r)}_{(i_r, j_r),\alpha_{r-1}\alpha_r} = f^{(r)}_{(k_r, i_r),\alpha_{r-1}\alpha_r} \delta_{l_rj_r}.
$$
This enables us to use efficient MPO-MPS multiplication implementations.

### Adaptive patching

To more efficiently compress the information we handle, we choose a strategy of adaptive patching by setting an upper bound $D_\mathrm{max}$ to the bond dimension $D$.
For illustration, picture a function $f(x, y)$ on a square domain $[0, 1]^2$.
We generate via TCI an MPS approximating $f$ on a $R \times R$ mesh with a certain measure of precision, discarding rows/columns of its tensors such that the result is equal to $f$ to within a given tolerance.
Now, if the MPS's bond dimension exceeds $D_\mathrm{max}$, we divide the domain into 4 smaller squares (or "patches") and construct an MPS on each with $R' \times R'$ where $R' = R - 1$, reusing the exact same interpolation points as the original domain.
We repeat this process, recursively subdividing our domain until the bond dimensions of all MPSs is smaller than or equal to $D_\mathrm{max}$.

The idea here is that in this way, we resolve more finely regions of the domain where $f$'s complexity is higher and are able to limit the bond dimension in areas of lower complexity, saving memory.
Of course, on the other hand, now multiplication becomes more involved.
Although they live on the same domain, multiplier and multiplicand are in general going to consist of different patches, so when multiplying we iterate over both sets of patches and test for overlapping patches which can then be multiplied as usual.

## Demonstration

We would now like to demonstrate the discussed techniques.
To this end, we choose the Hubbard Atom (############## TODO: intro Hubbard model?), where exact expressions for all quantities are known [@Thunström2018] and implemented in the package `HubbardAtoms.jl`.

A host of Julia packages is necessary:

In [102]:
import QuanticsGrids as QG     # utilities for handling quantics representations
using TensorCrossInterpolation # implementation of TCI
import TCIAlgorithms as TCIA   # implementation of patching
using HubbardAtoms             # exact results for the Hubbard atom
using SparseIR                 # provides the MatsubaraFreq types used in the HubbardAtoms package
using Quantics                 # high-level API for performing operations in QTT
using ITensors                 # efficient tensor computations and tensor network calculations

We can now use `QuanticsGrids.jl` to create a grid in the shape of a $R \times R \times R$ cube almost centered on the origin of "fermionic × fermionic × bosonic" Matsubara frequency space.
"Almost", because by construction the grid consists of an even number of points ($2^R$) in each direction and bosonic Matsubara frequencies are even, so a grid centered on the origin would include an odd number of frequencies.
This is something that will not affect our calculations in this example however, because we never sum over the bosonic axis.
If we wanted to do that -- e.g. in implementing the Schwinger-Dyson equation -- special care would need to be taken.

Later, interoperation between `TCIAlgorithms.jl` and `ITensors.jl` will be necessary, so we prepare the relevant `ITensors.Index` objects.
`zip`ing the indiviual axes' indices together corresponds to creating the indices in the fused representation introduce above.

In [156]:
function setup(R=4)
    N = 2^R
    grid = QG.InherentDiscreteGrid{3}(R, (-N + 1, -N + 1, -N); step=2, unfoldingscheme=:fused)

    sitesν = [Index(2, "ν=$r") for r in 1:R]
    sitesν´ = [Index(2, "ν´=$r") for r in 1:R]
    sitesω = [Index(2, "ω=$r") for r in 1:R]
    sitesfused = collect.(zip(sitesν, sitesν´, sitesω))
    sites = (; sitesν, sitesν´, sitesω, sitesfused)

    return grid, sites
end

setup (generic function with 3 methods)

Next, we use `HubbardAtoms.jl` to introduce the exact vertex functions $F$, $\Chi^0$ and $\Gamma$ and wrap them with `QuanticsGrids.quanticsfunction` to take a list of binary indices as input instead of the plain Matsubara frequencies.
Here, we also absorb the factor of $1 / \beta^2$ that appears in the BSE into the bare susceptibility for later convenience.

In [183]:
function makeverts(U, beta, ch, grid)
    model = HubbardAtom(U, beta)

    fq_full(x, y, z) = full_vertex(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    fq_chi0(x, y, z) = 1 / beta^2 * chi0(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    fq_gamma(x, y, z) = gamma(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    plainfuncs = (; fq_full, fq_chi0, fq_gamma)

    fI_full = QG.quanticsfunction(ComplexF64, grid, fq_full)
    fI_chi0 = QG.quanticsfunction(ComplexF64, grid, fq_chi0)
    fI_gamma = QG.quanticsfunction(ComplexF64, grid, fq_gamma)
    quanticsfuncs = (; fI_full, fI_chi0, fI_gamma)

    return plainfuncs, quanticsfuncs
end

makeverts (generic function with 1 method)

The so created functions are now adaptively TCIed into patched MPSs.
Because we work in 3 dimensions in the fused representation, the local dimensions are now $2^3 = 8$ in 

In [207]:
function interpolateverts(quanticsfuncs, maxbonddim, grid, sitesfused)
    (; fI_full, fI_chi0, fI_gamma) = quanticsfuncs
    
    localdims = dim.(sitesfused)
    projectable_full = TCIA.makeprojectable(Float64, fI_full, localdims)
    projectable_chi0 = TCIA.makeprojectable(Float64, fI_chi0, localdims)
    projectable_gamma = TCIA.makeprojectable(Float64, fI_gamma, localdims)
    
    pordering = TCIA.PatchOrdering(collect(1:grid.R))
    initialpivots = [QG.origcoord_to_quantics(grid, 0)] # approximate center of grid
    full_patches = TCIA.adaptiveinterpolate(projectable_full, pordering; verbosity=0, maxbonddim, initialpivots)
    chi0_patches = TCIA.adaptiveinterpolate(projectable_chi0, pordering; verbosity=0, maxbonddim, initialpivots)
    gamma_patches = TCIA.adaptiveinterpolate(projectable_gamma, pordering; verbosity=0, maxbonddim, initialpivots)
    
    sitedims = [dim.(s) for s in sitesfused]
    full_patches = reshape(full_patches, sitedims)
    chi0_patches = reshape(chi0_patches, sitedims)
    gamma_patches = reshape(gamma_patches, sitedims)

    patchesfuncs = (; full_patches, chi0_patches, gamma_patches)

    return patchesfuncs
end

interpolateverts (generic function with 2 methods)

In [208]:
function makevertsdiagonal(patchesfuncs, sites)
    (; full_patches, chi0_patches, gamma_patches) = patchesfuncs
    (; sitesν, sitesν´, sitesω, sitesfused) = sites

    sitesνν´_vec = [[x, y] for (x, y) in zip(sitesν, sitesν´)]
    sitesω_vec = [[z] for z in sitesω]
    sites_separateω = [x for pair in zip(sitesνν´_vec, sitesω_vec) for x in pair]

    full_mps = TCIA.ProjMPSContainer(Float64, full_patches, sitesfused)
    full_νν´_ω = Quantics.rearrange_siteinds(full_mps, sites_separateω)
    full_νν´_ωω´ = Quantics.makesitediagonal(full_νν´_ω, "ω")
    full_ptt = TCIA.ProjTTContainer{Float64}(full_νν´_ωω´)

    chi0_mps = TCIA.ProjMPSContainer(Float64, chi0_patches, sitesfused)
    chi0_νν´_ω = Quantics.rearrange_siteinds(chi0_mps, sites_separateω)
    chi0_νν´_ωω´ = Quantics.makesitediagonal(chi0_νν´_ω, "ω")
    chi0_νν´_ω´ω´´ = prime(chi0_νν´_ωω´)
    chi0_ptt = TCIA.ProjTTContainer{Float64}(chi0_νν´_ω´ω´´)

    gamma_mps = TCIA.ProjMPSContainer(Float64, gamma_patches, sitesfused)
    gamma_νν´_ω = Quantics.rearrange_siteinds(gamma_mps, sites_separateω)
    gamma_νν´_ωω´ = Quantics.makesitediagonal(gamma_νν´_ω, "ω")
    gamma_νν´_ω´´ω´´´ = prime(gamma_νν´_ωω´, 2)
    gamma_ptt = TCIA.ProjTTContainer{Float64}(gamma_νν´_ω´´ω´´´)

    diagonal_sites = full_νν´_ωω´.sites

    pttfuncs = (; full_ptt, chi0_ptt, gamma_ptt)

    return pttfuncs, diagonal_sites
end

makevertsdiagonal (generic function with 1 method)

In [209]:
function calculatebse(pttfuncs, diagonal_sites, grid, maxbonddim, sitesfused)
    (; full_ptt, chi0_ptt, gamma_ptt) = pttfuncs
    pordering = TCIA.PatchOrdering(collect(1:2(grid.R)))

    chi0_gamma_ptt = TCIA.adaptivematmul(chi0_ptt, gamma_ptt, pordering; maxbonddim)
    phi_bse_diagonal = TCIA.adaptivematmul(full_ptt, chi0_gamma_ptt, pordering; maxbonddim)
    phi_bse_diagonal_projmps = TCIA.ProjMPSContainer(Float64, phi_bse_diagonal, diagonal_sites)
    phi_bse_projmps_νν´_ω = Quantics.extractdiagonal(phi_bse_diagonal_projmps, "ω")
    phi_bse_projmps_νν´ω = Quantics.rearrange_siteinds(phi_bse_projmps_νν´_ω, sitesfused)
    phi_bse = TCIA.ProjTTContainer{Float64}(phi_bse_projmps_νν´ω)

    return phi_bse
end

calculatebse (generic function with 1 method)

In [210]:
function comparereference(phi_bse, plainfuncs, grid)
    N = 2^(grid.R)
    (; fq_full, fq_chi0, fq_gamma) = plainfuncs
    box = [
        (x, y, z) for x in range(-N + 1; step=2, length=N),
        y in range(-N + 1; step=2, length=N), z in range(-N; step=2, length=N)
    ]
    chi0_exact = map(splat(fq_chi0), box)
    full_exact = map(splat(fq_full), box)
    gamma_exact = map(splat(fq_gamma), box)
    phi_normalmul = stack(
        gamma_exact[:, :, i] * chi0_exact[:, :, i] * full_exact[:, :, i] for i in 1:N
    )

    phi_adaptivemul = [phi_bse(QG.origcoord_to_quantics(grid, p)) for p in box]

    return norm(phi_normalmul - phi_adaptivemul) / norm(phi_normalmul)
end

comparereference (generic function with 2 methods)

In [211]:
ch_d = DensityChannel()
ch_m = MagneticChannel()
ch_s = SingletChannel()
ch_t = TripletChannel()
channels = (ch_d, ch_m, ch_s, ch_t)

(DensityChannel(), MagneticChannel(), SingletChannel(), TripletChannel())

In [212]:
function main(U, beta, ch, R, maxbonddim)
    grid, sites = setup(R)
    plainfuncs, quanticsfuncs = makeverts(U, beta, ch, grid)
    patchesfuncs = interpolateverts(quanticsfuncs, maxbonddim, grid, sites.sitesfused)
    pttfuncs, diagonal_sites = makevertsdiagonal(patchesfuncs, sites)
    phi_bse = calculatebse(pttfuncs, diagonal_sites, grid, maxbonddim, sites.sitesfused)
    error = comparereference(phi_bse, plainfuncs, grid)
end

main (generic function with 1 method)

In [213]:
for ch in channels
    error = main(1.0, 1.3, ch, 4, 40)
    println(ch, ":\t", error)
end

DensityChannel():	0.0013234990449374465
MagneticChannel():	0.0006490663737673113
SingletChannel():	1.0
TripletChannel():	4.343290225596633e-15


In [101]:
using CairoMakie